# Model to Predict Elo Customer Loyalty

_Note! If you want to commit any changes to this document, please strip all output (Cell > Current Outputs > Clear, or set up [nbstripout](https://github.com/kynan/nbstripout) as a git filter) from this notebook before doing so. Thanks!_

## Unzip Data

First thing we do is look at the data that we have (you can download it with `make` in the project folder if you have a Kaggle API key set up locally), and unzipping the CSV files. We also need to change the permissions of the unzipped files, because by default it seems they are set to 000.

In [ ]:
!ls data/raw/

In [ ]:
!(cd data; for f in raw/*.zip; do unzip $f; done)

In [ ]:
!chmod 0644 data/*.csv

## Import Libraries

Next we import the Python libraries we'll need. If any of these are missing for you, you can install them with e.g. `pip3 install pandas` on the command line.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

## Load Data

Load the data into Pandas data frames and look at their structure.

In [ ]:
hist_trans_df = pd.read_csv('data/historical_transactions.csv')
merchants_df = pd.read_csv('data/merchants.csv')
merch_trans_df = pd.read_csv('data/new_merchant_transactions.csv')
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [ ]:
hist_trans_df.head()

In [ ]:
merchants_df.head()

In [ ]:
merch_trans_df.head()

In [ ]:
train_df.head()

## Explore Data

### Correlations

In [ ]:
sns.heatmap(train_df.corr(), vmin=-1, vmax=1, cmap='PiYG')

### Distributions

In [ ]:
sns.distplot(train_df.target)

In [ ]:
sns.countplot(x='feature_1', palette='Set2', data=train_df)

In [ ]:
sns.countplot(x='feature_2', palette='Set2', data=train_df)

In [ ]:
sns.countplot(x='feature_3', palette='Set2', data=train_df)

In [ ]:
sns.distplot(hist_trans.purchase_amount)